# Ice skating talent workshop

In [ ]:
## Leave your information so we may contact you
## These notebooks are deleted a few days after the workshop
## Feel free to only fill in the fields you want
YOUR_INFORMATION = dict(
    name="Patrick",
    phone_number="+31612324087",
    email="patrickacamara@gmail.com",
    feedback="your workshop feedback to us <3",
    study="AI",
    linkedin="https://www.linkedin.com/in/patrick-camara/"
)

# Goal

The goal of this workshop is to develop a machine learning tool that can **predict the performance of ice skaters**. Such a tool could be used to help identify the biggest talents in the field and aid coaches in selecting athletes for training and competition.

# Setup

First we set up our work environment. We use a bunch of packages and load our prepared datasets.

When you make changes to the notebook, it is often best to use the `Run All >>` button at the top. We've done our best to make sure running the entire notebook is quite fast.

## Shortcuts

- `Shift+Enter` to execute a cell - you can spam this to continue running cells
- In the left sidebar, the third entry is a `Navigation` tab. Use it to quickly navigate the notebooks.
- Drag the tab of a notebook to split the view
- In the top-bar -> Settings -> Theme -> Jupyter Dark

### Tools

We import some specific packages for this workshop:

- [pandas](https://pandas.pydata.org/docs/index.html):  A widely used data structures and data analysis package for processing our data as DataFrames;
- [Jupyter](https://jupyter.org/):  The workspace you are working in right now, to easily create notebooks for experimenting with data, models, and visualisations;
- [scikit-learn](https://scikit-learn.org/stable/index.html ):  To create model pipelines combining transformers and estimators;
- [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_intro.html): A widely used prediction model for both classification and regression that often needs little tuning to quickly get decent results;


Of course you are free to use other machine learning libraries as well, for example PyTorch or Tensorflow,.

In [2]:
import sys, os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["figure.figsize"] = (20, 10)
warnings.filterwarnings('ignore')

### Data

Qualogy manages the historical database for the KNSB. For every skater we have historical records about all races they participated in, as well as data about the skater like their nationality, gender and age.

#### Features

Our **features** should capture the historical performance of a skater. To summarize the **history of a skater at a given age**, we take all their data from before that age and summarize it. Our goal is to create features such that we have **1 record for person_id + season_age**. To prevent data-leakage, we may not use any data of the year we are trying to predict the performance for.

#### Target

Our **target**s are the **future season_best records**, so we can predict how good a skater will be in the future - given his performance until the current age. For every age we predict multiple years ahead.

### Feature Engineering
The data we have prepared for you contains both static features and timeseries features.
 - **Static features**: data that does not change over time, such as [`birthdates`, `name`, `gender`]
 - **Timeseries features**: this data changes over time, and is aggregated by yearly seasons.
   - Aggregations over the historical data of the skaters, which includes but not limited to:
       - `season_best` to represent the performance for every yearly season
       - `n_races_season` for the number of races participated in during a season
   - Summary of the performance of the previous years of an individual skater.
   - This is what is shown in the figure below.

![image.png](https://tsfresh.readthedocs.io/en/latest/_images/rolling_mechanism_1.png)!
![image.png](https://tsfresh.readthedocs.io/en/latest/_images/rolling_mechanism_2.png)!

In the figures above, the contributors to [TSFresh](https://tsfresh.readthedocs.io/en/latest/text/forecasting.html) illustrate how we calculate rolling features to predict a target. In this illustration **only one-step ahead targets are shown**. For this workshop, **we create targets for ALL future records**.

## Get the train/test data

In [20]:
X_train = pd.read_pickle("data/X_train.pkl")
X_test = pd.read_pickle("data/X_test.pkl")
y_train = pd.read_pickle("data/y_train.pkl")
y_test = pd.read_pickle("data/y_test.pkl")



nan_count_col1 = X_train.isna().sum()
print(nan_count_col1)


   person_id  season_age  season_age_target  personal_best  season_best  \
0       3681          14                 15          48.12        48.12   
1       3681          14                 16          48.12        48.12   
2       3681          14                 17          48.12        48.12   
3       3681          14                 18          48.12        48.12   
4       3681          14                 19          48.12        48.12   

   std_season_best  std_season_best_3_years  season_avg_time  \
0              NaN                      NaN           50.915   
1              NaN                      NaN           50.915   
2              NaN                      NaN           50.915   
3              NaN                      NaN           50.915   
4              NaN                      NaN           50.915   

   1_year_improvement  2_year_improvement  ...  \
0                 NaN                 NaN  ...   
1                 NaN                 NaN  ...   
2             

In [ ]:
print(X_train.head())
print(y_train)

X_train.describe()
y_train.describe()

## Display data

At every prediction moment, we create features based on all the known information until that point. Then we use those features to predict all known future `season_best` values. Of course, these values are only known in our training data.

The real use of this model is to apply it to current young talent to predict their unkown future performance!

# Create model

The **FUN** part!

### 1. Create scikit-learn transformers
Can must instantiate a transformer from scikit-learn before using it in the model.
Here we create some transformers for you to start with. 

We want you to experiment with the Transformers, and their hyperparameters.
You can add any other transformers if you wish: https://scikit-learn.org/stable/data_transforms.html

### 2. Create the estimator using regression

### 3. Create your pipeline by combining transformers and your estimator
A model consists of a pipeline of Transformers that are performed sequentially and a final step that is an Estimator (xgboost in this case).
 - Add/remove more transformers by adding a tuple of ('name', transformer_object) or commenting out any of the steps below;
 - You can also change the XGBRegressor to use a different estimator but for this workshop we do not recommend it.


### 4. (Alternative) Build a machine learning model with PyTorch / Tensorflow.
Instead of using regression models, you are free to create a different approach, such as building a neural network .

## Hands-on

**Here we want you to edit the cells below to change and experiment with your model pipeline.**

Sklearn docs: [https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

In [12]:
# Experiment and create your own model.
import sklearn
from sklearn import linear_model, decomposition
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, OneHotEncoder
from sklearn.linear_model import LinearRegression
from xgboost import plot_importance, XGBRegressor

from sklearn.ensemble import RandomForestRegressor


from sklearn import set_config
set_config(display="diagram")

# You can group the prediction features into various lists to run different
# This helps us to treat different types of variables differently.
# Feel free to make changes to this list by including/excluding other columns from `X_train`.

all_features = X_train.columns.values

print(all_features)

['person_id' 'season_age' 'season_age_target' 'personal_best'
 'season_best' 'std_season_best' 'std_season_best_3_years'
 'season_avg_time' '1_year_improvement' '2_year_improvement' 'season'
 'n_races_season' 'n_races_total' 'n_races_3_year' 'years_of_experience'
 'total_races_national_junior_championships'
 'total_races_world_junior_championships' 'total_races_junior_world_cups'
 'total_races_world_cups' 'season_races_national_junior_championships'
 'season_races_world_junior_championships'
 'season_races_junior_world_cups' 'season_races_world_cups' 'gender'
 'nationality' 'year_of_birth']


## Create model pipeline

The following cell shows a baseline model to predict the ice skaters performance in future years.
You can make changes to the pipeline, for example swapping the estimator or adding/removing steps in the pipeline.

Edit the cells below to create a new model pipeline.

- Pick / Divide the features;
- Add / Remove / Combine transformers;
- Make your final step an `Estimator`; such as LinearRegression or an XGBoost Regressor
- For each transformer you can tweak the parameters.

In [10]:
# # We selected a few features to train the model on
# subset_features_1 = [
#     'season_age',
#     'season_age_target',
#     'season_best',
#     'year_of_birth',
#     'n_races_season',
# ]

# subset_features_2 = [
#     'year_of_birth',
#     'years_of_experience',
# ]

# # two simple pipelines, that use the features from above
# pipeline1 = Pipeline([
#     ("scaler", MinMaxScaler()),
#     ("pca", PCA(n_components=5))
# ])

# pipeline2 = Pipeline([
#     ("standard_scaler", StandardScaler(with_mean=True, with_std=False)),
#     ("imputer", SimpleImputer(strategy="constant", fill_value=0, add_indicator=False)),
# ])

# # An estimator as example
# estimator = LinearRegression()

# # complete the pipeline by adding all steps together
# model = Pipeline([
#     ("column_transformer", ColumnTransformer(
#         transformers=[
#             ("pipeline_1", pipeline1, subset_features_1),
#             ("pipeline_2", pipeline2, subset_features_2),
#         ]
#     )),
#     ("estimator", estimator)
# ])

# model

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('pipeline_1',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler()),
                                                                  ('pca',
                                                                   PCA(n_components=5))]),
                                                  ['season_age',
                                                   'season_age_target',
                                                   'season_best',
                                                   'year_of_birth',
                                                   'n_races_season']),
                                                 ('pipeline_2',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler(with_std=False)),
                                                                  ('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant'))]),
                                                  ['year_of_birth',
                                                   'years_of_experience'])])),
                ('estimator', LinearRegression())])

# Train models

- Train the model by calling `model.fit(X_train, y_train)`;
- Predict with the model by calling `model.predict(X_test)`;


In [27]:
"""
Explanation of Model Setup and Decisions:

1. Feature Selection:
   - 'subset_features_1' focuses on direct performance metrics and improvements:
     'personal_best' and 'season_best' are crucial as they directly indicate the skater's best times.
     'std_season_best' and 'std_season_best_3_years' provide insights into the variability of performances, 
     which can help understand consistency and potential under varying conditions.
     'season_avg_time', '1_year_improvement', and '2_year_improvement' are included to capture recent trends 
     and changes in performance, offering predictive clues about future improvements or regressions.

   - 'subset_features_2' combines experience and demographic features:
     'years_of_experience' and 'n_races_total', 'n_races_3_year' quantify the amount of competitive exposure 
     a skater has, which is often correlated with their skill level and adaptability.
     'gender' and 'nationality' are included to potentially capture demographic influences on performance,
     acknowledging that certain trends might exist within different groups or national sports systems.

2. Preprocessing Pipelines:
   - The 'numerical_pipeline' includes:
     'SimpleImputer' with a strategy of 'mean' to handle missing values in numerical data by replacing them with 
     the mean of their respective columns, ensuring no data point is discarded due to missing values.
     'StandardScaler' normalizes these features to have zero mean and unit variance, which is important for models 
     like PCA and many machine learning algorithms to perform optimally.
     'PCA' is used to reduce the dimensionality of the data to the three most informative components, simplifying 
     the model while retaining the most significant information, which helps in improving model performance and 
     reducing overfitting.

   - The 'categorical_pipeline' includes:
     'SimpleImputer' with a strategy of 'most frequent' to fill missing values in categorical data with the most 
     common category, preserving the statistical properties of the feature.
     'OneHotEncoder' transforms categorical variables into a numerical format that the model can utilize, 
     expanding the feature space to include binary variables that represent the presence or absence of each category.

3. Column Transformer:
   - This tool applies the specified 'numerical_pipeline' to 'subset_features_1' and 'categorical_pipeline' to 
     'subset_features_2', ensuring that each feature subset is processed according to its nature and requirements.
     The 'remainder' parameter set to 'passthrough' allows any other features not explicitly mentioned to still 
     be included in the model without alteration, providing flexibility.

4. Model Pipeline:
   - Integrates preprocessing and modeling steps into a single pipeline, streamlining the process from raw data 
     to predictions. This setup uses 'RandomForestRegressor', a robust and versatile ensemble method that can handle 
     both linear and non-linear relationships effectively. It is chosen for its ability to model complex interactions 
     between features and its general robustness to overfitting and various data imperfections.

5. Model Training and Evaluation:
   - The model is trained using the processed features from 'X_train' and the target 'y_train'. After training, 
     it is evaluated on 'X_test' to calculate the Mean Squared Error (MSE) from the predictions compared to 'y_test'.
     This MSE provides a quantitative measure of the model's prediction accuracy in the same units as the target variable.

This setup is designed to be comprehensive yet efficient, ensuring that each type of data is handled appropriately 
and that the model benefits from both raw performance data and contextual demographic information to provide accurate 
and reliable predictions.
"""
print()

In [17]:
subset_features_1 = [
    'personal_best', 'season_best', 'std_season_best', 'std_season_best_3_years',
    'season_avg_time', '1_year_improvement', '2_year_improvement'
]

subset_features_2 = [
    'years_of_experience', 'n_races_total', 'n_races_3_year',
    'gender', 'nationality'
]



# Define more comprehensive preprocessing for numerical and categorical data
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),  # Impute missing values with the mean of the column
    ("scaler", StandardScaler()),  # Scale data to have zero mean and unit variance
    ("pca", PCA(n_components=3))   # Reduce dimensionality to focus on the most informative aspects
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Impute missing values with the most frequent category
    ("encoder", OneHotEncoder(handle_unknown='ignore'))  # Convert categorical variables into a format that can be provided to the model
])

# Define column transformer to apply different preprocessing to different subsets of data
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, subset_features_1),  # Apply numerical_pipeline to first subset of features
        ("cat", categorical_pipeline, subset_features_2)  # Apply categorical_pipeline to second subset of features
    ],
    remainder='passthrough'  # Pass through other features not specified in subsets without any changes
)

# Define the complete model pipeline
model = Pipeline([
    ("preprocessor", preprocessor),
    ("estimator", RandomForestRegressor(n_estimators=100, random_state=42))  # Using RandomForestRegressor as the model
])

# Example of how to fit the model (assuming X_train and y_train are already loaded)
model.fit(X_train, y_train)

# Example of how to evaluate the model (assuming X_test and y_test are already loaded)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.647088689489535


In [23]:
import numpy as np

rmse = np.sqrt(3.647088689489535)
mean_target = y_train.mean()
median_target = np.median(y_train)

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean of target variable: {mean_target}')
print(f'Median of target variable: {median_target}')

Root Mean Squared Error: 1.9097352406785437
Mean of target variable: 40.693063173072595
Median of target variable: 40.3


In [28]:
"""
Changes in the Model Setup:

1. Numerical Data Preprocessing:
   - 'SimpleImputer' strategy changed from 'mean' to 'median':
     This change helps handle outliers more effectively, as the median is less affected by extreme values compared to the mean.
   - 'RobustScaler' replaces 'StandardScaler':
     RobustScaler is used instead of StandardScaler to reduce the impact of outliers. It scales features using the median and the interquartile range, making it suitable for data with outliers.

2. No changes in Categorical Data Preprocessing:
   - The handling of categorical data remains the same with 'SimpleImputer' using 'most frequent' to fill missing values and 'OneHotEncoder' to handle categorical features.

3. Model Pipeline:
   - The overall structure of the model pipeline has not changed. It includes a preprocessor for handling both numerical and categorical data and uses 'RandomForestRegressor' as the estimator.
   
These adjustments make the model more robust to outliers and irregularities in the data, which is important for maintaining accuracy in real-world applications.
"""
print()

In [19]:
# Define preprocessing for numerical data
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # Use median for imputation to handle outliers
    ("scaler", RobustScaler()),  # Use RobustScaler to reduce the influence of outliers
    ("pca", PCA(n_components=3))  # Use PCA for dimensionality reduction
])

# Define preprocessing for categorical data
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Impute missing categories with the most frequent value
    ("encoder", OneHotEncoder(handle_unknown='ignore'))  # OneHotEncode categorical variables
])

# Define column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, subset_features_1),
        ("cat", categorical_pipeline, subset_features_2)
    ],
    remainder='passthrough'
)

# Define the model pipeline
model = Pipeline([
    ("preprocessor", preprocessor),
    ("estimator", RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit the model (assuming X_train, y_train are already loaded)
model.fit(X_train, y_train)

# Predict and evaluate the model (assuming X_test, y_test are already loaded)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 3.7056169680228876


In [24]:
rmse = np.sqrt(3.7056169680228876)
mean_target = y_train.mean()
median_target = np.median(y_train)

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean of target variable: {mean_target}')
print(f'Median of target variable: {median_target}')

Root Mean Squared Error: 1.9249979137710482
Mean of target variable: 40.693063173072595
Median of target variable: 40.3


In [32]:
"""
Justifications for Using XGBoost in the Model Pipeline:

1. Handling of Missing Values:
   - XGBoost has an in-built capability to handle missing values automatically. This reduces the need for extensive data preprocessing and can lead to better handling of gaps in data.

2. Regularization Features:
   - XGBoost includes both L1 and L2 regularization which helps in reducing overfitting. Regularization is critical in complex models to ensure that they generalize well on unseen data.

3. Model Flexibility:
   - The ability to define custom objective functions and tuning a wide range of hyperparameters makes XGBoost highly adaptable. This flexibility allows it to be fine-tuned for specific dataset characteristics, such as the varying scales and distributions in sports performance data.

4. Scalability and Efficiency:
   - XGBoost is designed to be highly efficient and scalable, making it suitable for models that need to handle large volumes of data with complex relationships.

5. Enhanced Tree Modeling:
   - XGBoost uses a more sophisticated approach to building trees, which can model non-linear relationships more effectively than models like linear regression and can be more robust than random forests under certain configurations.

Setting Parameters:
   - 'n_estimators': Controls the number of boosting stages which should be increased or tuned based on the model performance and overfitting.
   - 'learning_rate': Helps in controlling the contribution of each tree, making the model more robust by reducing the risk of overfitting.
   - 'max_depth': Helps in managing the complexity of each tree, where deeper trees capture more detailed information.

By leveraging these features, XGBoost can potentially offer superior performance on datasets with complex patterns and varied data quality, making it a strong candidate for comparison against other models.
"""
print()


In [29]:
# Define preprocessing for numerical data
numerical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),  # Impute missing values using median
    ("scaler", StandardScaler())  # Scale features to zero mean and unit variance
])

# Define preprocessing for categorical data
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Impute missing values with the most common category
    ("encoder", OneHotEncoder(handle_unknown='ignore'))  # OneHotEncode categorical variables
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, subset_features_1),
        ("cat", categorical_pipeline, subset_features_2)
    ],
    remainder='passthrough'
)

# XGBoost model configuration
xgb_model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42))
])

# Fit the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f'Mean Squared Error for XGBoost: {mse_xgb}')

Mean Squared Error for XGBoost: 2.9627747234977755


In [30]:
rmse = np.sqrt(2.9627747234977755)
mean_target = y_train.mean()
median_target = np.median(y_train)

print(f'Root Mean Squared Error: {rmse}')
print(f'Mean of target variable: {mean_target}')
print(f'Median of target variable: {median_target}')

Root Mean Squared Error: 1.721271252155736
Mean of target variable: 40.693063173072595
Median of target variable: 40.3


In [50]:
# !pip install lightgbm

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import StackingRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [54]:
lgbm_params = {
    'n_estimators': 200,  # Increase number of boosting rounds
    'learning_rate': 0.05,  # Lower learning rate for more robust learning
    'num_leaves': 31,  # Default, consider adjusting based on performance
    'max_depth': -1,  # No limit, but consider setting a limit if overfitting occurs
    'min_data_in_leaf': 20,  # Increase if model is overfitting
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Tree
    'objective': 'regression',
    'random_state': 42  # For reproducibility
}

# Define features explicitly
categorical_features = ['gender', 'nationality']
numeric_features = [col for col in X_train.columns if col not in categorical_features]

# Preprocessing for numeric data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the models in the stack
estimators = [
    ('xgboost', XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3, enable_categorical=True)),
    ('lightgbm', LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)),
    ('mlp', Pipeline([
        ('scaler', StandardScaler()),  # Ensure scaling is applied for MLP
        ('mlp', MLPRegressor(hidden_layer_sizes=(50,), activation='relu', solver='adam', max_iter=500))
    ]))
]

# Create the stacking ensemble
stack_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    passthrough=False,
    cv=5
)

# Apply preprocessing and fit the model
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('stack', stack_model)
])

# Fit the model
full_pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = full_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003630 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2700
[LightGBM] [Info] Number of data points in the train set: 22193, number of used features: 34
[LightGBM] [Info] Start training from score 40.693063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [ ]:
# # train the model
# model = model.fit(X_train, y_train)

# # use your model to make predictions
# yhat = model.predict(X_test)

# # evaluate your code according to metrics by your choosing.
